<a href="https://colab.research.google.com/github/alexwangliting/MachineLearning2Submission_WangLiTing/blob/main/11_SWIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!gdown 10HrFogT1PuROJ2XLWn7NKQa-DnSoiLGC

Downloading...
From (original): https://drive.google.com/uc?id=10HrFogT1PuROJ2XLWn7NKQa-DnSoiLGC
From (redirected): https://drive.google.com/uc?id=10HrFogT1PuROJ2XLWn7NKQa-DnSoiLGC&confirm=t&uuid=cec24724-7fd4-472a-96d1-1ef8f8d828af
To: /content/oxfpetsdata.zip
100% 1.13G/1.13G [00:11<00:00, 101MB/s]


In [ ]:
!unzip /content/oxfpetsdata.zip

Streaming output truncated to the last 5000 lines.
  inflating: oxfpetsdata/images/Birman_107.png  
  inflating: __MACOSX/oxfpetsdata/images/._Birman_107.png  
  inflating: oxfpetsdata/images/basset_hound_172.png  
  inflating: __MACOSX/oxfpetsdata/images/._basset_hound_172.png  
  inflating: oxfpetsdata/images/basset_hound_166.png  
  inflating: __MACOSX/oxfpetsdata/images/._basset_hound_166.png  
  inflating: oxfpetsdata/images/newfoundland_180.png  
  inflating: __MACOSX/oxfpetsdata/images/._newfoundland_180.png  
  inflating: oxfpetsdata/images/shiba_inu_156.png  
  inflating: __MACOSX/oxfpetsdata/images/._shiba_inu_156.png  
  inflating: oxfpetsdata/images/Birman_113.png  
  inflating: __MACOSX/oxfpetsdata/images/._Birman_113.png  
  inflating: oxfpetsdata/images/saint_bernard_159.png  
  inflating: __MACOSX/oxfpetsdata/images/._saint_bernard_159.png  
  inflating: oxfpetsdata/images/Egyptian_Mau_113.png  
  inflating: __MACOSX/oxfpetsdata/images/._Egyptian_Mau_113.png  
  inflati

In [ ]:
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, random_split
import torchvision.transforms.v2 as transforms
from torch.utils.tensorboard import SummaryWriter
from timm.models import create_model

In [ ]:
from torch.utils.data import Dataset
import os
import json
from functools import partial
from torchvision.io import read_image
import torch
def get_img(path):
    return read_image(path).to(torch.float32) / 255
def get_num(num):
    return num
class OxfordPets(Dataset):
    def __init__(self, root, target_types, transform=None):
        self.root = root
        self.target_types = target_types
        self.transform = transform
        self._data = []
        if root is None:
            root = "/content/oxfpetsdata"
        with open(os.path.join(root, "data.json"), "r") as f:
            for k, v in json.load(f).items():
                targets = {
                    "category": partial(get_num, v["category"]),
                    "species_id": partial(get_num, v["species_id"]),
                }
                self._data.append((
                    partial(get_img, os.path.join(root, "images", f"{k}.png")),
                    *(targets[t] for t in target_types),
                ))

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        img, *targets = self._data[idx]
        img = img()  # Call the partial function to load the image
        targets = tuple(t() for t in targets)  # Evaluate any partial functions for targets
        if self.transform:
            img = self.transform(img)
        return img, targets[0] if len(targets) == 1 else targets

transform = transforms.Compose([
    transforms.Resize((224, 224), antialias=True),
    transforms.Lambda(lambda x: x * 2 - 1),
])

data = OxfordPets(None, target_types=["category"], transform=transform)


In [ ]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 128
EPOCHS = 2
SEED = 42

MODEL_NAME = "swin_tiny_patch4_window7_224"  # Example Swin Transformer model from timm
BLOCK_DROP_P = 0.0
ATTN_DROP_P = 0.0

MODEL_OUT = "model.pth"
DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)



def evaluate(writer, step, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    writer.add_scalar("Loss/Test", test_loss, step)
    writer.add_scalar("Accuracy/Test", correct, step)
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )


def train_one_epoch(writer, step, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(DEVICE), y.to(DEVICE)

        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    writer.add_scalar("Loss/Train", train_loss, step)
    writer.add_scalar("Accuracy/Train", correct, step)
    print(
        f"Train Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n"
    )


def main():
    torch.manual_seed(SEED)


    transform = transforms.Compose(
        [
            transforms.Resize((224, 224), antialias=True),
            transforms.Lambda(lambda x: x * 2 - 1),
        ]
    )

    data = OxfordPets(None, target_types=["category"], transform=transform)
    unique_classes = set()  # Initialize an empty set to hold unique classes

    for _, label in data:
        unique_classes.add(label)  # Directly add the label, assuming it's already an integer

    print("Unique classes in dataset:", unique_classes)
    train_data, test_data = random_split(data, [int(0.8 * len(data)), len(data) - int(0.8 * len(data))])

    train_sampler = RandomSampler(train_data)
    test_sampler = SequentialSampler(test_data)

    data_loader_train = DataLoader(
        train_data,
        batch_size=BATCH_SIZE,
        sampler=train_sampler,
    )
    data_loader_test = DataLoader(
        test_data,
        batch_size=BATCH_SIZE,
        sampler=test_sampler,
    )


    model = create_model(
    MODEL_NAME,  # Specify the desired Swin Transformer model name
    pretrained=True,
    block_drop_rate=BLOCK_DROP_P,
    attn_drop_rate=ATTN_DROP_P,
)

    model.head.fc.out_features = 37  # Adapt output layer to match number of classes
    model.to(DEVICE)


    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()
    writer = SummaryWriter()


    for epoch in range(EPOCHS):
        print(f"Epoch {epoch+1}\n-------------------------------")
        train_one_epoch(writer, epoch, data_loader_train, model, loss_fn, optimizer)
        evaluate(writer, epoch, data_loader_test, model, loss_fn)
    writer.close()
    print("Done!")


if __name__ == "__main__":
    main()


Unique classes in dataset: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Epoch 1
-------------------------------
loss: 7.454885  [  128/ 2936]
loss: 6.416615  [  256/ 2936]
loss: 5.935493  [  384/ 2936]
loss: 5.427165  [  512/ 2936]
loss: 5.199770  [  640/ 2936]
loss: 4.723072  [  768/ 2936]
loss: 4.585427  [  896/ 2936]
loss: 4.366721  [ 1024/ 2936]
loss: 4.158666  [ 1152/ 2936]
loss: 4.095001  [ 1280/ 2936]
loss: 3.647208  [ 1408/ 2936]
loss: 3.799472  [ 1536/ 2936]
loss: 3.500361  [ 1664/ 2936]
loss: 3.380680  [ 1792/ 2936]
loss: 3.096800  [ 1920/ 2936]
loss: 2.982769  [ 2048/ 2936]
loss: 2.655835  [ 2176/ 2936]
loss: 2.563182  [ 2304/ 2936]
loss: 2.523372  [ 2432/ 2936]
loss: 2.229334  [ 2560/ 2936]
loss: 2.177696  [ 2688/ 2936]
loss: 1.960542  [ 2816/ 2936]
loss: 1.721444  [ 2760/ 2936]
Train Error: 
 Accuracy: 20.1%, Avg loss: 3.852240 

Test Error: 
 Accuracy: 68.8%, Avg loss: 1.181764 

Epoch 2
-------------------------------
loss: 1.339739  [  128/ 2936]
loss: 1.187914  [  256/ 2936]
loss: 1.219540  [  384/ 2936]
loss: 0.929531  [  512/ 2936]
loss: